In [2]:
#!/usr/bin/python

from ase.build import fcc111
from ase import Atoms
from ase.io import write,read
from ase.visualize import view
import numpy as np
import math

# alpha, d & Nx, Ny are the lattice parameter (bulk), interplanar spacing and later dimensions (Nx x Ny xlayers)
alpha=4.1983
d = alpha/(3**0.5)
Nx = 6  # use either even or odd 
Ny = 6  # only use even values because we will setup orthorombic cells

# junction layers considering PBC (sum of bottom and upper)
layers= 6

#The following loop enables usage of odd number of layers e.g., 5 layers = 3 for bottom and 2 for top electrode
if (layers % 2) == 0:
    Nz = layers
else:
    Nz = layers + 1
       
# Defining the lower base electrode orthogonally
elec = fcc111('Au', size=(Nx,Ny,Nz), vacuum=None, a=alpha, orthogonal=True)

# extracting cell parameters useful for later 
unit = elec.get_cell()

# cuts the upper layers of the electrode; (x-1)*d means x layers are kept; done this way for symmetry reasons
Z = ((Nz/2)-1)*d
del elec[elec.positions[:, 2] > Z]

N_z = int(Nz/2)
elec2 = fcc111('Au', size=(Nx,Ny,N_z), vacuum=None, a=alpha, orthogonal=True)

# translate the 2nd electrode in the z-direction by an amount = size of water box you want later; 20 Å used here
water_Zsize = 20
if (layers % 2) == 0:
    elec2.translate([0, 0, (water_Zsize+2*d)])
else:
    del elec2[elec2.positions[:, 2] < d/2 ]
    elec2.translate([0, 0, (water_Zsize+2*d)])

# This combines both electrodes giving your system  
system = elec + elec2

# generate proper unit cell parameters for system, ref denotes Z-position of last atom; used to get C unit cell vector 
ref = elec2.get_positions()[(len(elec2.get_positions()) - 1), :]
system.set_cell([[unit[0, 0], unit[0, 1], unit[0, 2]], [unit[1, 0], unit[1, 1], unit[1, 2]], [0.0, 0.0, (d+ref[2])]])

# get proper indices
system.set_tags(range(len(system)))

# to check if correct uncomment below
view(system)

# extract structure file
write('electrodes.xyz', system)

# now we generate packmol script to solvate our system
# we define our water box dimensions as follows: inside box xmin ymin zmin xmax ymax zmax
xmin = 0
ymin = 0
R = (alpha/(2*math.sqrt(2)))  #radius of metal atom
zmin = ((Nz/2) - 1)*d + R + 2.2

xmax = unit[0, 0] - 2
ymax = unit[1, 1] - 2
zmax = ((Nz/2) - 1)*d + water_Zsize - R - 2.2

rho = 1*10**(-24) # g/Å3
NA = 6.022*10**23 # avogadro constant
Mr = 18 # molecular mass of water

# for volume, we subtract 1/2 spheres from box volume because water molecules may seep in between spheres
V = unit[0, 0]*unit[1, 1]*(water_Zsize) - 2*((Nx*Ny)*(1/2)*(4/3)*math.pi*(R**3))

#number of water molecules
No_W = round((rho*V*NA)/Mr)

packmol = open('packmol.inp', 'w')
print("output solvated_system.xyz",\
      "\nfiletype xyz",\
      "\nstructure electrodes.xyz",\
      "\n   fixed 0 0 0 0 0 0",\
      "\nend structure",\
      "\nstructure water.xyz",\
      "\n   number", No_W,\
      "\n   tolerance 2.0",\
      "\n   inside box", xmin, ymin, zmin, xmax, ymax, zmax,\
      "\nend structure", file= packmol)
packmol.close()